In [ ]:
# coding: utf-8
import os
import time
import sys
import tables
from types import SimpleNamespace
import pickle
import pandas as pd
import numpy as np
import torch
from torch import nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.backends import cudnn 
#Visualization
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams

#Custom code
from student import Student
from feedforwardcompressed import FeedforwardCompressed, get_batch, slice_blocks, block_speed, timeit_speed
from evaluator import Evaluator
import dataloaders
from randomized_svd import torch_randomized_svd
from tools import plot_sigma, save_V_array, load_V_array, save_inv_ind
from tools import load_inv_ind, compute_approx, HaveInterlayers, dump_svd, create_folders
from models import vgg19, cifar10, cifar100, stl10
from models.cifar import vgg19 as vgg19_100
from train_validate import train, validate, adjust_learning_rate, save_checkpoint

print('Should be > 1.0.0 : {}'.format(torch.__version__))
cudnn.benchmark = True

In [ ]:
config = SimpleNamespace()
config.dir_path = 
config.storage_path = 
config.data_root = 

config.device = 'cuda'
config.cuda_device = 2
config.batch_size = 128

config.dataset = 'cifar10'
config.model = 'vgg19'
# vgg19, cifar10, cifar100, stl10

#Steps of experiments:
#For fast test one can use precomputer SVD's (check supported models):
config.download_svd = False
#Or calculate by themself (set download to false:
config.calculate_interlayers = False
config.compute_svd = False
config.validate_svd = True
config.plot_singular_values = True
config.calc_compressed = True
config.eval_compressed = True
config.calc_finetune = True
config.measure_speed = True
config.vis_time_accuracy = True

config.skip_if_computed = True

#SVD step additional configuration
config.svd_rank = 10000
config.svd_maxval = 10000
config.svd_nsamples = [80000]*20

#Compression step additional configuration
config.compression_block_num_list = [2,3,4,5,6]
config.compression_rate = 0.1

#Fine-tune additional configuration
config.ft_variance = 0.5
config.kd = False
# - Adam's parameters
optim_parameters = SimpleNamespace()
optim_parameters.lr0 = 1e-4*0.5
optim_parameters.weight_decay = 5e-4
optim_parameters.momentum = 0.9
optim_parameters.start_epoch = 0
optim_parameters.epochs = 90
# - KD loss's parameters (add loss from teacher)
kd_params = None
if config.kd:
    kd_params = SimpleNamespace()
    kd_params.alpha = 0.001
    kd_params.temperature = 3.5
    
#Speed meter parameters:
config.speed_device = 'cpu'
config.speed_batchsize = 1
print('Use config: {}'.format(config))

In [ ]:
sys.path.append(config.dir_path)
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=str(config.cuda_device)

create_folders(config)
#Create dataloaders
loaders = dataloaders.get_loader(config.batch_size, config.dataset, data_root=config.data_root,
                                     num_workers=4, drop_last=True)

#Load pretrained model
print('Loading pretrained initial model {}\n'.format(config.model))
if config.model == 'vgg19' and config.dataset == 'cifar100':
    name = config.dataset+'/'+config.model
    if not os.path.exists(config.storage_path+'pretrained/'+config.dataset+'/'
                    +config.model+'/'+'model_best.pth.tar'):
        print('please, pu')
    model = vgg19_100(num_classes = 100,
                     pretrained = config.storage_path+'pretrained/'+config.dataset+'/'
                    +config.model+'/'+'model_best.pth.tar')
    
if config.model == 'vgg19' and config.dataset == 'cifar10':
    name = config.dataset+'/'+config.model
    if not os.path.exists(config.storage_path+'pretrained/'+config.dataset+'/'
                    +config.model+'/'+'model_best.pth.tar'):
        print('please, pu')
    model = vgg19(pretrained = config.storage_path+'pretrained/'+config.dataset+'/'
                    +config.model+'/'+'model_best.pth.tar')

if config.model == 'playground' and config.dataset == 'cifar10':
    if not os.path.exists(config.storage_path+'pretrained/'+config.dataset+'/'
                    +config.model+'/'+'model_best.pth.tar'):
        print('please, pu')
    model = cifar10(n_channel=128, pretrained = True)

elif config.model == 'svhn':
    name = ['svhn/']        
    model = [(models.svhn(32, pretrained=True))]
    
#Create compression list for choosen model:
tmp, _ = slice_blocks(model, loaders['val'], config.device)
n = len(tmp)
config.compression_rate_list = []
config.teacher=[None]*n
config.compression_rate_list.append(config.teacher)
for k in config.compression_block_num_list: 
    compression_rate =  [None]*(n - k) + [config.compression_rate] * k 
    config.compression_rate_list.append(compression_rate)      
config.finetune_compression_rate_list = config.compression_rate_list

In [ ]:
if not config.download_svd:
    #You need a lot of RAM on two following steps, so, you also can download SVD
    # Interlayer outputs are needed for SVD
    if config.calculate_interlayers:
        print('Calculating and store outputs after each block in model ...\n')    

        tmp_path = config.storage_path+'interlayers/'+config.dataset+'/'+config.model
#         if len(os.listdir(tmp_path)):
#             print('Directory {} is not empty, skipping calculation.\n'.format(tmp_path))
        if 1:
            #Add method that return interlayer outputs
            new_model = HaveInterlayers(model.cuda(), loaders['train'])
            #Environment that has method of load data to model and return interlayers
            env = Evaluator(new_model, loaders)
            iteration = 0
            #min(n_features, N_samples) x n_features
            B = []
            max_iteration = max(config.svd_nsamples)/config.batch_size

            for A in env.get_interlayer():
                if iteration == 0:
                    for j in range(len(A)):
                        size_to_log = min(int(np.ceil(A[j].shape[1]/config.batch_size)*config.batch_size), 
                                          int(config.svd_nsamples[j]/config.batch_size)*config.batch_size)
                        if size_to_log == 0:
                            continue
                        B.append(np.zeros((size_to_log, A[j].shape[1]), dtype=np.float32))
                        B[j][:config.batch_size,:] = A[j]
                        max_iteration = max(max_iteration, B[j].shape[0]/config.batch_size)
                elif iteration >= max_iteration:
                    break
                else:
                    for i in range(len(A)):
                        if (config.batch_size*(iteration+1)) < min(B[i].shape):
                            B[i][config.batch_size*iteration:config.batch_size*(iteration+1),:] = A[i]
                iteration += 1

            for i in range(len(B)):
                h5file = tables.open_file(tmp_path+'/{}.h5'.format(i), mode='w')
                root = h5file.root
                h5file.create_array(root, 'B'+str(i), B[i])
                h5file.close()  
    if config.compute_svd:
        dump_svd(config.dataset+'/'+config.model, config.storage_path+'interlayers/', max_r=config.svd_rank, k=50)
else:
    tmp_path = config.storage_path+'interlayers/'+config.dataset+'/'+config.model
    if not os.path.exists(tmp_path+'/svd/'):
        os.mkdir(tmp_path+'/svd/')
    

In [ ]:
if config.validate_svd:
    tmp_path = config.storage_path+'interlayers/'+config.dataset+'/'+config.model
    for i in range(1,7):
        g_t = tmp_path+'/{}.h5'.format(i)
        filename = tmp_path+'/svd/SVD_after_block_{}.pickle'.format(i)
        with open(filename, 'rb') as f:
            (U,S,V) = pickle.load(f)
            print(U.shape, V.shape)
        with h5py.File(g_t, mode='r') as f:
            for key,val in f.items():
                X=torch.cuda.DoubleTensor(val[()])
                print(X.shape)
        print(X)
        new_X = (U[:, ]@torch.diag(S)@V.t())
        print(new_X)
        new_X = torch.cuda.DoubleTensor(new_X.cuda())
        print(torch.norm(X-new_X))

In [ ]:
if config.plot_singular_values:    
    tmp_path = config.storage_path+'interlayers/'+config.dataset+'/'+config.model
    results_path = config.storage_path+'results/'+config.dataset+'/'+config.model

    files = [f for f in os.listdir(tmp_path+'/svd/') if os.path.isfile(os.path.join(tmp_path+'/svd/', f))]
    #save figures of sorted sigmas
    if not os.path.exists(results_path+'/svd_imgs/'):
        os.mkdir(results_path+'/svd_imgs/')
    f = plt.figure(figsize=(1.35*3.25, 0.6*1.35*3.25))
    legend = []
    for i in range(len(files)):
        filename = tmp_path+'/svd/SVD_after_block_{}.pickle'.format(i)
        legend.append(i)
        with open(filename, 'rb') as f:
            (U,S,V) = pickle.load(f)
            s = np.sort(S.cpu().numpy())[::-1][:min(V.shape[0], config.svd_maxval)-150]
            s = s/s[0]
        plt.semilogy(s)
    plt.title(config.model.upper()+' on '+config.dataset.upper())
    plt.ylabel('$\sigma$')
    plt.savefig(results_path+'/svd_imgs/'+config.model.upper()+'on'+config.dataset.upper()+'.pdf', bbox_inches='tight')
    plt.show()


In [ ]:
if config.calc_compressed:                    
    W_shapes, block_indices = slice_blocks(model, loaders['val'], config.device)
    tmp_path = config.storage_path+'interlayers/'+config.dataset+'/'+config.model
    SVD_list = []
    for idx, W in enumerate(W_shapes):
        filename = tmp_path + '/svd/SVD_after_block_{}.pickle'.format(idx)
        with open(filename, 'rb') as f:
            U, S, V = pickle.load(f)
        SVD_list += [(U.numpy(), S.numpy(), V.numpy())]
    
    if not os.path.exists(results_path+'/compression_results.txt'):
        os.remove(results_path+'/compression_results.txt')
    
     
    for compression_rate in config.compression_rate_list:
        print(compression_rate)

        compressed_model = FeedforwardCompressed(model, loaders['val'], compression_rate, 'cpu')
        print('Fitting...')
        compressed_model.fit(loaders['val'], SVD_list)

        print('Creating student...')
        student = Student(compressed_model, device = config.device)

        student.to('cpu')
        save_path = config.storage_path+'compressed/'+config.dataset+'/'+\
                    config.model+'/{}.pth'.format(compression_rate)
        torch.save(student, save_path)
        
        results_path = config.storage_path+'results/'+config.dataset+'/'+config.model
        
        if config.eval_compressed:
            print_info = {
             'dataset': config.dataset,
             'batch_size' : config.batch_size,
             'arch': config.model,
             'compression_rate': compression_rate,
             'filename': results_path+'/compression_results.txt'  
            }
            validate(loaders['val'], student, print_info, device=config.device, is_svhn=(config.model=='svhn'))



In [ ]:
if config.calc_finetune:
    tmp_path = config.storage_path+'compressed/'+config.dataset+'/'+config.model
    results_path = config.storage_path+'results/'+config.dataset+'/'+config.model
    if not os.path.exists(tmp_path+'/finetuned/'):
        os.mkdir(tmp_path+'/finetuned/')

    teacher_model = torch.load(tmp_path+'/{}.pth'.format(config.teacher), map_location='cpu')

    for compression_rate in config.finetune_compression_rate_list:
        best_prec1 = 0
        precs = np.arange(10)         
        print('Starting finetune model: {}'.format(compression_rate))
        model_path = tmp_path+'/{}.pth'.format(compression_rate)
        ft_model_path = tmp_path+'/finetuned/{}'.format(compression_rate)
        ft_model = torch.load(model_path, map_location=config.device)

        #Add Dropout before compressed layers
        for i,m in enumerate(ft_model.features):
            if 'Flatten' in str(m):
                ft_model.features[i] = nn.Sequential(ft_model.features[i])
                ft_model.features[i].add_module('drop', nn.Dropout())

        optimizer = torch.optim.Adam(ft_model.parameters(), optim_parameters.lr0,
            weight_decay=optim_parameters.weight_decay)

        print('Optimizer: {}'.format(optimizer))
        #Dir for finetuned model
        if not os.path.exists(ft_model_path):
            os.mkdir(ft_model_path)
        #CSV with training results
        with open(results_path+'/'+str(compression_rate)+'/finetune_results.csv', 'w') as f:
            f.write('{},{},{},{},{},{},{},{}\n'.format('epoch', 'epoch_time', 'train@1', 'train@5', 'train loss', 'val@1', 'val@5', 'val loss', ))

        for epoch in range(optim_parameters.start_epoch, optim_parameters.epochs):

            lr = adjust_learning_rate(optimizer, epoch, lr0=optim_parameters.lr0)
            print('Epoch: [{}/{}]\tlr: {}'.format(epoch, optim_parameters.epochs, lr))
            # train for one epoch
            end = time.time()
            train(loaders['train'], ft_model, teacher_model, optimizer, epoch, kd_params, config.device, is_svhn=(config.model=='svhn'))
            epoch_time = time.time()-end
            # evaluate on validation set
            print('Train:\n')
            train_top1, train_top5, train_losses = validate(loaders['train'], ft_model, device=config.device, is_svhn=(config.model=='svhn'))
            print('Val:\n')
            top1, top5, losses = validate(loaders['val'], ft_model, device=config.device, is_svhn=(config.model=='svhn'))
            prec1=top1.avg
            # remember best prec@1 and save checkpoint
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            precs[int(epoch%10)] = best_prec1

            with open(results_path+'/'+str(compression_rate)+'/finetune_results.csv', 'a') as f:
                f.write('{},{},{},{},{},{},{},{}\n'.format(epoch + 1, epoch_time, train_top1.avg, train_top5.avg, train_losses.avg,
                                               top1.avg, top5.avg, losses.avg))

            save_checkpoint({
                'epoch': epoch + 1,
                'arch': name,
                'state_dict': ft_model.state_dict(),
                'best_prec1': best_prec1,
                'optimizer' : optimizer.state_dict()
            }, is_best, ft_model_path)
            print (np.var(precs))
            if np.var(precs) < config.ft_variance:
                print('STABLE @ {}\n'.format(epoch))
                break

In [ ]:
if config.measure_speed:
     
    for compression_rate in config.compression_rate_list:
        print(compression_rate)
        results_path = config.storage_path+'results/'+config.dataset+'/'+config.model
        save_path = config.storage_path+'compressed/'+config.dataset+'/'+\
                    config.model+'/{}.pth'.format(compression_rate)
        student = torch.load(save_path)
        student.to(config.speed_device)
        student.eval()
        speedloaders = dataloaders.get_loader(config.speed_batchsize, config.dataset, config.data_root, num_workers=4)
        result_df = pd.DataFrame()
        times_block = block_speed(student, speedloaders['val'], config.speed_device)
        whole_timeit = timeit_speed(student, speedloaders['val'], config.speed_device)
        result_df['Block'] = times_block
        result_df['Whole'] = whole_timeit+[0]*(len(times_block)-1)
        result_df.to_pickle(results_path+'/{}_{}.pickle'.format(compression_rate, device))
        print('DONE: %d'%i)
        torch.cuda.empty_cache()

In [ ]:
if config.vis_time_accuracy:
    def show_results(path):
        with open(path, 'r') as f:
            info = f.readlines()

        columns = ['dataset', 'arch', 'compression_rate', 'batch_size',
           'losses.avg', 'losses.get_var()', 'top1.avg', 'top5.avg',
           'batch_time.avg', 'batch_time.get_var()', 'device']
        info_data = pd.DataFrame([line.strip().split('\t') for line in info], columns=columns)

        for column in columns:
            if column not in ['dataset', 'arch', 'compression_rate', 'device']:
                info_data[column] = info_data[column].astype('float')

        info_rate = []
        for l in info_data['compression_rate']:
            rates = list(filter(lambda el: el!='None', [el.strip() for el in l.strip('[,]').split(',')]))
            rates = np.array(rates, dtype=float)

            if len(rates) != 0:
                info_rate.append([len(rates), rates[0]])
            else:
                info_rate.append([len(rates), 1])

        info_data[['N_compressed_blocks', 'rate']] =  pd.DataFrame(info_rate)
        return info_data  
    
    
    def get_acc_speed(config, compression_rate):
        results_path = config.storage_path+'results/'+config.dataset+'/'+config.model       
        finetune_path = config.storage_path+'compressed/'+config.dataset+'/'+config.model+\
                        '/finetuned/{}/'.format(compression_rate)
        best_epoch = torch.load(finetune_path+'model_best.pth.tar')['epoch']
        acc_df = pd.read_csv(results_path + '/' + str(compression_rate)+'/finetune_results.csv')
    
        if os.path.exists(results_path + '/{}_{}.pickle'.format(compression_rate, config.speed_device)):
            with open(results_path + '/{}_{}.pickle'.format(compression_rate, config.speed_device), 'rb') as f:
                df_column = pickle.load(f)
        
        total_time = df_column['Whole'][0]

        info_data = show_results(results_path + '/compression_results.txt')
        info_data_dataset = info_data[info_data['dataset']==config.dataset]
        row = info_data_dataset[info_data_dataset['compression_rate']==str(compression_rate)]
        print(acc_df)
        return total_time.average, acc_df.loc[best_epoch-1, 'val@1'], acc_df.loc[best_epoch-1, 'val@5'], row['top1.avg'], row['top5.avg']
    

    rc('font',**{'family':'serif','sans-serif':['Computer Modern Roman']})
    rcParams.update({'font.size': 10})
    f = plt.figure(figsize=(1.35*3.25, 1.35*0.6*3.25))
    #Teacher first
    t, prec_ft, prec_ft5, prec, prec5 = get_acc_speed(config, config.teacher)
    original_time = t
    speed = 1.
    plt.scatter(1, prec_ft, marker='o', c=(0.1, 0.2, 0.5))
    plt.axvline(x=1, linestyle='--', color=(0.1, 0.2, 0.5), lw=0.5)
    plt.axhline(y=prec_ft, linestyle='--', color=(0.1, 0.2, 0.5), lw=0.5)
    prec_ft_bl = prec_ft

    #Lists for figure
    color_list = ['{},{},{}'.format(0.1, 0.2, 0.5)]   
    accuracy_list_before = [prec]
    accuracy_list_after = [prec_ft]
    accuracy_list_before5 = [prec5]
    accuracy_list_after5 = [prec_ft5]
    speed_up_list = [speed]

    for i, compression_rate in enumerate(config.finetune_compression_rate_list):
        t, prec_ft, prec_ft5, prec, prec5 = get_acc_speed(config, compression_rate)
        speed = original_time/t
        plt.scatter(speed, prec_ft, marker='o', c=(0.3*i%1, 0.2*i%1, 0.1*i%1))
        plt.scatter(speed, prec, marker='^', c=(0.3*i%1, 0.2*i%1, 0.1*i%1))

        color_list.append('{},{},{}'.format(0.3*i%1, 0.2*i%1, 0.1*i%1))    
        accuracy_list_before.append(prec)
        accuracy_list_after.append(prec_ft)
        accuracy_list_before5.append(prec5)
        accuracy_list_after5.append(prec_ft5)
        speed_up_list.append(speed)

    plt.xlabel('Speed up')
    plt.ylabel('Accuracy, %')
    plt.xlim([0.9, 3.1])
    plt.ylim(ylims[dataset])
    plt.title('Speed vs Accuracy for '+'{}'.format(dataset).upper())
    plt.savefig(config.storage_path+'results/'+config.dataset+'/'+config.model+'/speed_accuracy_{}.pdf'.format(dataset), bbox_inches='tight')
    # plt.legend(names, loc=10)
    plt.show()

    table = pd.DataFrame()
    beauty_names = ['Teacher'] + ['{:.1f}x {:d} blocks'.format(1/x.max(), int(sum(x!=None))) 
                    for x in config.finetune_compression_rate_list]

    table['Name'] = beauty_names
    table['Color'] = color_list
    table['Prec@1 \\newline before FT'] = accuracy_list_before
    table['Prec@5 \\newline before FT'] = accuracy_list_before5

    table['Prec@1 \\newline after FT'] = accuracy_list_after
    table['Prec@5 \\newline after FT'] = accuracy_list_after5

#     table['Speed up conv part'] = speed_up_list
    table['Speed up'] = speed_up_list
    table = table.sort_values(by=['Speed up'])
    print(table)